# CUHK [STAT3009](https://www.bendai.org/STAT3009/) Notebook1: Baseline methods for Recommender Systems

## Software prepare
- `Code Editor`: VS Code; Sublime; or Atom

- `Terminal`: Iterm2 in Mac; Deepin terminal in Linux

## Creating virtual environments
- If you have multiple versions of Python on your system, you can select a specific Python version by running python3 or whichever version you want.

- To create a virtual environment, decide upon a directory where you want to place it, and run the venv module as a script with the directory path:

- How to create and activate a virtual environment, see Section 12.2 in the [Document]((https://docs.python.org/3/tutorial/venv.html)).

- Install packages via `pip`, see `Installing packages` section in the [Document]((https://docs.python.org/3/tutorial/venv.html)).


## Kaggle competition about recommender systems: user and item can be extended to more general cases.
- [Elo Merchant Category Recommendation](https://www.kaggle.com/c/elo-merchant-category-recommendation/data?select=Data+Dictionary.xlsx): `merchant_id` and `card_id`.

- [WSDM - KKBox's Music Recommendation Challenge](https://www.kaggle.com/c/kkbox-music-recommendation-challenge/data): `user` and `music`.

- [Event Recommendation Engine Challenge](https://www.kaggle.com/c/event-recommendation-engine-challenge/overview/evaluation): `user` and `event`.

## Load dataset into Python
- Dowload [Netflix Prize Data](https://www.kaggle.com/netflix-inc/netflix-prize-data). (For illustration, we only take the first subset.)

- Load data into Python.

- Re-orginize the data structure as a standard form.

- For testing set, we hide the real ratings.

- We only take the first subset for illustration.

In [1]:
import numpy as np
import pandas as pd

dtrain = pd.read_csv('./dataset/train.csv')
dtrain.sample(5)

,movie_id,user_id,rating,date
37984,3333,1960,4,2003-12-08
40504,3081,135,4,2004-12-03
7029,3094,538,5,2005-01-19
27699,267,1591,5,2004-09-22
11851,1389,983,4,2004-11-20


In [2]:
dtest = pd.read_csv('./dataset/test.csv')
## save real ratings for test set for evaluation.
test_rating = np.array(dtest['rating'])
## remove the ratings in the test set to simulate prediction
dtest = dtest.drop(columns='rating')
dtest.sample(5)

,movie_id,user_id,date
15170,602,968,2003-06-10
14265,2828,103,2005-09-09
31870,1297,360,2005-08-21
42554,769,425,2003-05-09
13153,2716,1894,2005-02-07


## Pre-process the data as a `np.array`

In [3]:
## convert string to user_id and item_id -> [user_id, item_id, rating]
# pre-process for training data
train_pair = dtrain[['user_id', 'movie_id']].values
train_rating = dtrain['rating'].values
# pre-process for testing set
test_pair = dtest[['user_id', 'movie_id']].values

n_user, n_item = max(train_pair[:,0].max(), test_pair[:,0].max())+1, max(train_pair[:,1].max(), test_pair[:,1].max())+1

## Implement Baseline methods: global\_average, user\_average and item\_average (For your practice)
- Inpout: training set.

- Output: return predicted ratings for (user id, item id) user-item pairs in testing set.

- Goal: make prediction for testing set

In [4]:
pred = np.zeros(len(test_rating))

In [5]:
## Global average
global_pred = pred.copy()
global_mean = train_rating.mean()
global_pred = global_mean*np.ones(len(pred))
print(global_pred[:10])

[3.62115674 3.62115674 3.62115674 3.62115674 3.62115674 3.62115674
 3.62115674 3.62115674 3.62115674 3.62115674]


### user\_average
- Loop for all users
    - Find all records for this user in both training and testing sets.
    - Compute the average ratings for this user in the training set.
    - Predict the ratings for this users in the testing set.

In [6]:
## user average
UA_pred = pred.copy()
for u in range(n_user):
    # find the index for both train and test for user_id = u
    ind_test = np.where(test_pair[:,0] == u)[0]
    ind_train = np.where(train_pair[:,0] == u)[0]
    if len(ind_test) == 0:
        continue
    if len(ind_train) < 3:
        UA_pred[ind_test] = global_mean
    else:
        # predict as user average
        UA_pred[ind_test] = train_rating[ind_train].mean()
print(UA_pred[:10])

[3.73684211 3.35714286 3.66037736 2.84931507 3.70909091 3.27419355
 3.16666667 3.57142857 4.1        3.375     ]


## Evaluation: compute RMSE for baseline methods
- Input: (1) predicted testing ratings (2) real testing ratings

- Output: RMSE for the prediction

- Goal: evaluate the prediction performance for the method.

In [7]:
## RMSE for Global average
rmse_glb = np.sqrt(np.mean((global_pred - test_rating)**2))
print('RMSE for GLB average: %.3f' %rmse_glb)

RMSE for GLB average: 1.085


In [8]:
## RMSE for user average
rmse_usr = np.sqrt(np.mean((UA_pred - test_rating)**2))
print('RMSE for user average: %.3f' %rmse_usr)

RMSE for user average: 1.013


## Summarize `glb_average` and `user_average` methods as Python functions

### `glb_average`

- *Input*: 'train_rating', 'test_pair'

- *Return*: Predicted ratings based on glb mean.

In [11]:
def glb_mean(train_rating, test_pair):
    pred = train_rating.mean() * np.ones(len(test_pair))
    return pred

### `user_average`

- *Input*: 'train_pair', 'train_rating', 'test_pair'

- *Return*: Predicted ratings based on user mean.

In [12]:
def user_mean(train_pair, train_rating, test_pair):
    n_user = max(train_pair[:,0].max(), test_pair[:,0].max())+1
    pred = np.zeros(len(test_pair))
    glb_mean_value = train_rating.mean()
    for u in range(n_user):
        # find the index for both train and test for user_id = u
        ind_test = np.where(test_pair[:,0] == u)[0]
        ind_train = np.where(train_pair[:,0] == u)[0]
        if len(ind_test) == 0:
            continue
        if len(ind_train) < 3:
            pred[ind_test] = glb_mean_value
        else:
            # predict as user average
            pred[ind_test] = train_rating[ind_train].mean()
    return pred

In [ ]:
def item_mean(train_pair, train_rating, test_pair):
    n_item = max(train_pair[:,1].max(), test_pair[:,1].max())+1
    pred = np.zeros(len(test_pair))
    glb_mean_value = train_rating.mean()
    for i in range(n_item):
        # find the index for both train and test for item_id = i
        ind_test = np.where(test_pair[:,1] == i)[0]
        ind_train = np.where(train_pair[:,1] == i)[0]
        if len(ind_test) == 0:
            continue
        if len(ind_train) < 3:
            pred[ind_test] = glb_mean_value
        else:
            # predict as user average
            pred[ind_test] = train_rating[ind_train].mean()
    return pred

## Summarize `Evaluation` as a Python function

In [13]:
def rmse(true, pred):
    return np.sqrt(np.mean((pred - true)**2))